In [12]:
from checklist.test_suite import TestSuite
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd

In [ ]:
def get_summary_info(suite):
    passed = 0
    failed = 0
    for test_name in suite.tests:
        table = suite.visual_summary_by_test(test_name)
        
        failed += table.stats['nfailed']    
        passed += table.stats['npassed']
        assert table.stats['nfailed'] + table.stats['npassed'] == len(table.filtered_testcases)
    if passed+failed > 0:
        return {
            "failed": failed,
            "failed(%)": (failed/(passed+failed))*100,
            "passed": passed,
            "passed(%)": (passed/(passed+failed))*100,
            "total": passed+failed,
            "templates": len(suite.tests)
        }
    else:
        return {
            "failed": 0,
            "failed(%)": 0,
            "passed": 0,
            "passed(%)": 0,
            "total": 0,
            "templates": 0
        }

# with spacy

In [8]:
suite_apps = {
    "approach1": TestSuite.from_file('./suites/posneg-approach1.suite'),
    "approach2": TestSuite.from_file('./suites/posneg-approach2.suite'),
    "approach3": TestSuite.from_file('./suites/posneg-approach3.suite'),
    "approach4": TestSuite.from_file('./suites/posneg-approach4.suite'),
    "approach5": TestSuite.from_file('./suites/posneg-approach5.suite'),
    "random": TestSuite.from_file('./suites/posneg-random.suite')
}

In [35]:
df_list = []
columns = ["approach","failed","failed(%)","passed","passed(%)","total","templates"]

for k, v in suite_apps.items():
    # df_list.append()
    # print()
    df_list.append([k] + list(get_summary_info(v).values()))

df_with_spacy = pd.DataFrame(df_list, columns=columns)

In [37]:
df_with_spacy

,approach,failed,failed(%),passed,passed(%),total,templates
0,approach1,0,0.000000,0,0.000000,0,0
1,approach2,0,0.000000,0,0.000000,0,0
2,approach3,269,17.332474,1283,82.667526,1552,19
3,approach4,269,17.332474,1283,82.667526,1552,19
4,approach5,732,23.529412,2379,76.470588,3111,26
5,random,164,26.623377,452,73.376623,616,18


# with nltk

In [38]:
suite_apps_nltk = {
    "approach1": TestSuite.from_file("../notebooks_rotten_tomatoes/suites/posneg-approach1.suite"),
    "approach2": TestSuite.from_file('../notebooks_rotten_tomatoes/suites/posneg-approach2.suite'),
    "approach3": TestSuite.from_file('../notebooks_rotten_tomatoes/suites/posneg-approach3.suite'),
    "approach4": TestSuite.from_file('../notebooks_rotten_tomatoes/suites/posneg-approach4.suite'),
    "approach5": TestSuite.from_file('../notebooks_rotten_tomatoes/suites/posneg-approach5.suite'),
    "random": TestSuite.from_file('../notebooks_rotten_tomatoes/suites/posneg-random.suite')
}

In [39]:
df_list = []
columns = ["approach","failed","failed(%)","passed","passed(%)","total","templates"]

for k, v in suite_apps_nltk.items():
    # df_list.append()
    # print()
    df_list.append([k] + list(get_summary_info(v).values()))

df_with_nltk = pd.DataFrame(df_list, columns=columns)

In [40]:
df_with_nltk

,approach,failed,failed(%),passed,passed(%),total,templates
0,approach1,1,50.000000,1,50.000000,2,1
1,approach2,1,16.666667,5,83.333333,6,2
2,approach3,139,10.570342,1176,89.429658,1315,20
3,approach4,126,10.994764,1020,89.005236,1146,19
4,approach5,372,16.467463,1887,83.532537,2259,25
5,random,51,8.173077,573,91.826923,624,18


In [45]:
# approaches = df_ann1_summ["approach"].map(lambda x: "FN   |   FP   |    FN     |      FP</br></br> Ann1      |       Ann2</br>" + x[:-5])
approaches = df_with_spacy["approach"]

def trunc(text_iter):
    return text_iter.map(lambda x: f"{x:.1f}")


fig = go.Figure(data=[
    go.Bar(name="Passing (Spacy)", x=approaches, y=df_with_spacy["passed(%)"]/100, text=trunc(df_with_spacy["passed(%)"]), textposition=None, marker_color=px.colors.qualitative.T10[1]),
    go.Bar(name="Passing (NLTK)", x=approaches, y=df_with_nltk["passed(%)"]/100, text=trunc(df_with_nltk["passed(%)"]), textposition=None, marker_color=px.colors.qualitative.Set3[5]),
    go.Bar(name="Failing (Spacy)", x=approaches, y=df_with_spacy["failed(%)"]/100, text=trunc(df_with_spacy["failed(%)"]), textposition=None, marker_color=px.colors.qualitative.T10[0]),
    go.Bar(name="Failing (NLTK)", x=approaches, y=df_with_nltk["failed(%)"]/100, text=trunc(df_with_nltk["failed(%)"]), textposition=None, marker_color=px.colors.qualitative.Set3[4]),
])
fig.update_layout(
    title_text=f'[Rotten Tomatoes] Passing x Failing comparing Spacy and NLTK',
    barmode="group",
    yaxis = dict(
        range=[0, 1],
        tickformat = '.0%',
        ),
    xaxis = dict(
        tickvals=[0,1,2,3,4,5],
        ticktext=approaches,
        ),
    )
fig.show()